In [5]:
import functions_class as fx
import classx as cl
import log_reg_functions as lrf
import loaddata as ld



import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss

# Load data

In [110]:
x,y = ld.load_data(scaler='minmax')

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.25, random_state=42, shuffle=True)

# Egen logistic regression.


In [116]:
eta = 0.1 # This is out eta

Niteration = 1000
beta = np.random.randn(x.shape[1],1)
costvec=[]
loglvec=[]
yaxis=[]



for iter in range(Niteration):
    
    sig = lrf.sigmoid(xtrain@beta)
    gradient = lrf.gradient_ols(xtrain,ytrain,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(xtrain@beta,ytrain.T)
    cost_test = lrf.cost_log_ols(xtest@beta,ytest.T)
    # Log Loss function from sklearn
    logloss=log_loss(ytrain, np.round(xtrain@beta), eps=1e-16, normalize=True)
    print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_test)
    #costvec.append(cost.ravel())
    #loglvec.append(logloss)
    #yaxis.append(iter+1)
    #plt.plot(costvec)
    #plt.plot(loglvec)
    
    
   

Cost [[0.7909796]] & Log loss 9.701430143213068 & Cost test [[0.78878912]]
Cost [[0.78755855]] & Log loss 9.68509338198937 & Cost test [[0.78563681]]
Cost [[0.78420415]] & Log loss 9.662207045390042 & Cost test [[0.78254682]]
Cost [[0.78091491]] & Log loss 9.63771584501026 & Cost test [[0.7795177]]
Cost [[0.77768939]] & Log loss 9.623007183326626 & Cost test [[0.77654802]]
Cost [[0.77452612]] & Log loss 9.590356896638834 & Cost test [[0.77363634]]
Cost [[0.77142368]] & Log loss 9.570773230638846 & Cost test [[0.77078126]]
Cost [[0.76838063]] & Log loss 9.565888932018655 & Cost test [[0.76798138]]
Cost [[0.76539558]] & Log loss 9.556069216107135 & Cost test [[0.76523532]]
Cost [[0.76246713]] & Log loss 9.54458422344018 & Cost test [[0.76254173]]
Cost [[0.75959391]] & Log loss 9.529870914604626 & Cost test [[0.75989927]]
Cost [[0.75677458]] & Log loss 9.529866267452704 & Cost test [[0.75730664]]
Cost [[0.75400781]] & Log loss 9.516794999612976 & Cost test [[0.75476254]]
Cost [[0.7512923]

Cost [[0.61219127]] & Log loss 8.621454976958923 & Cost test [[0.62250973]]
Cost [[0.61142795]] & Log loss 8.616556736882966 & Cost test [[0.62177288]]
Cost [[0.61067161]] & Log loss 8.614919343039059 & Cost test [[0.62104228]]
Cost [[0.60992215]] & Log loss 8.613291243498995 & Cost test [[0.62031785]]
Cost [[0.60917947]] & Log loss 8.610021102963103 & Cost test [[0.6195995]]
Cost [[0.6084435]] & Log loss 8.60675560957913 & Cost test [[0.61888715]]
Cost [[0.60771415]] & Log loss 8.610021102963103 & Cost test [[0.61818073]]
Cost [[0.60699132]] & Log loss 8.605113568583302 & Cost test [[0.61748017]]
Cost [[0.60627493]] & Log loss 8.59693124651569 & Cost test [[0.61678538]]
Cost [[0.60556491]] & Log loss 8.595293852671782 & Cost test [[0.61609629]]
Cost [[0.60486116]] & Log loss 8.595279911216018 & Cost test [[0.61541284]]
Cost [[0.60416362]] & Log loss 8.59854075744807 & Cost test [[0.61473495]]
Cost [[0.60347219]] & Log loss 8.591995829224365 & Cost test [[0.61406255]]
Cost [[0.6027868]

Cost [[0.55277878]] & Log loss 8.240151333165075 & Cost test [[0.56311064]]
Cost [[0.55246372]] & Log loss 8.23851858647309 & Cost test [[0.56278108]]
Cost [[0.55215044]] & Log loss 8.23851858647309 & Cost test [[0.56245318]]
Cost [[0.55183891]] & Log loss 8.235243798785277 & Cost test [[0.56212694]]
Cost [[0.55152911]] & Log loss 8.233601757789447 & Cost test [[0.56180234]]
Cost [[0.55122104]] & Log loss 8.228680281953887 & Cost test [[0.56147935]]
Cost [[0.55091466]] & Log loss 8.22213535373018 & Cost test [[0.56115797]]
Cost [[0.55060998]] & Log loss 8.215585778354551 & Cost test [[0.56083819]]
Cost [[0.55030696]] & Log loss 8.213953031662564 & Cost test [[0.56051998]]
Cost [[0.55000559]] & Log loss 8.209045497282766 & Cost test [[0.56020333]]
Cost [[0.54970586]] & Log loss 8.202495921907138 & Cost test [[0.55988823]]
Cost [[0.54940776]] & Log loss 8.19594169937959 & Cost test [[0.55957467]]
Cost [[0.54911125]] & Log loss 8.186131277771912 & Cost test [[0.55926263]]
Cost [[0.5488163

Cost [[0.5257353]] & Log loss 7.845655595581549 & Cost test [[0.53415336]]
Cost [[0.52554476]] & Log loss 7.842385455045656 & Cost test [[0.53394489]]
Cost [[0.52535492]] & Log loss 7.840757355505591 & Cost test [[0.53373714]]
Cost [[0.52516576]] & Log loss 7.835849821125792 & Cost test [[0.53353008]]
Cost [[0.52497729]] & Log loss 7.8342124272818845 & Cost test [[0.53332372]]
Cost [[0.5247895]] & Log loss 7.8276628519062585 & Cost test [[0.53311805]]
Cost [[0.52460238]] & Log loss 7.826030105214272 & Cost test [[0.53291307]]
Cost [[0.52441593]] & Log loss 7.822755317526458 & Cost test [[0.53270878]]
Cost [[0.52423015]] & Log loss 7.816210389302753 & Cost test [[0.53250517]]
Cost [[0.52404503]] & Log loss 7.816210389302753 & Cost test [[0.53230223]]
Cost [[0.52386057]] & Log loss 7.814572995458846 & Cost test [[0.53209997]]
Cost [[0.52367676]] & Log loss 7.811293560619111 & Cost test [[0.53189838]]
Cost [[0.5234936]] & Log loss 7.808014125779376 & Cost test [[0.53169745]]
Cost [[0.5233

Cost [[0.50735395]] & Log loss 7.580709152047357 & Cost test [[0.51383529]]
Cost [[0.50722202]] & Log loss 7.579071758203449 & Cost test [[0.51368823]]
Cost [[0.50709047]] & Log loss 7.577434364359544 & Cost test [[0.51354158]]
Cost [[0.50695929]] & Log loss 7.57907640535537 & Cost test [[0.51339532]]
Cost [[0.50682849]] & Log loss 7.577443658663387 & Cost test [[0.51324946]]
Cost [[0.50669805]] & Log loss 7.577443658663387 & Cost test [[0.51310401]]
Cost [[0.50656798]] & Log loss 7.577443658663387 & Cost test [[0.51295894]]
Cost [[0.50643827]] & Log loss 7.577448305815307 & Cost test [[0.51281428]]
Cost [[0.50630893]] & Log loss 7.5758109119714 & Cost test [[0.51267001]]
Cost [[0.50617996]] & Log loss 7.57255006573935 & Cost test [[0.51252613]]
Cost [[0.50605134]] & Log loss 7.569279925203458 & Cost test [[0.51238264]]
Cost [[0.50592309]] & Log loss 7.570912671895443 & Cost test [[0.51223954]]
Cost [[0.50579519]] & Log loss 7.570921966199284 & Cost test [[0.51209683]]
Cost [[0.5056676

Cost [[0.49461252]] & Log loss 7.494312991929755 & Cost test [[0.49957663]]
Cost [[0.49451501]] & Log loss 7.494312991929755 & Cost test [[0.49946712]]
Cost [[0.49441775]] & Log loss 7.494317639081675 & Cost test [[0.4993579]]
Cost [[0.49432074]] & Log loss 7.492680245237769 & Cost test [[0.49924896]]
Cost [[0.49422399]] & Log loss 7.492680245237769 & Cost test [[0.49914029]]
Cost [[0.49412749]] & Log loss 7.494317639081677 & Cost test [[0.4990319]]
Cost [[0.49403123]] & Log loss 7.495955032925582 & Cost test [[0.49892379]]
Cost [[0.49393523]] & Log loss 7.495955032925582 & Cost test [[0.49881596]]
Cost [[0.49383948]] & Log loss 7.495959680077504 & Cost test [[0.49870839]]
Cost [[0.49374397]] & Log loss 7.494326933385518 & Cost test [[0.49860111]]
Cost [[0.49364871]] & Log loss 7.494326933385518 & Cost test [[0.49849409]]
Cost [[0.4935537]] & Log loss 7.49433158053744 & Cost test [[0.49838735]]
Cost [[0.49345894]] & Log loss 7.49433158053744 & Cost test [[0.49828088]]
Cost [[0.49336442

KeyboardInterrupt: 

# Accuracy. 
Både egen kode og tester med scikit. 

In [114]:

activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'%')


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'%')


81.44888888888889 %
81.48 %


In [115]:


model = LogisticRegression()
model.fit(xtrain, ytrain)
predicted_classes = model.predict(xtrain)
accuracy = accuracy_score(ytrain.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytrain, predicted_classes)

print(accuracy, '% for training data')

predicted_classes = model.predict(xtest)
accuracy = accuracy_score(ytest.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytest, predicted_classes)

print(accuracy, '% for test data')

C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


82.19111111111111 % for training data
81.85333333333334 % for test data


In [80]:
#self.X_data_full = X_data 
#self.Y_data_full = Y_data

n_inputs = X.shape[0]
n_features = X.shape[1]

#self.epochs = epochs
#self.batch_size = batch_size
#self.iterations = self.n_inputs // self.batch_size
#self.eta = eta
#self.lmbd = lmbd



n_hidden_neurons=50  # velge andre verdier om vi vil
n_categories=2
epochs=100
batch_size=80
eta=0.0001
lmbd=0.1


hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_bias = np.zeros(n_hidden_neurons) + 0.01

output_weights = np.random.randn(n_hidden_neurons, n_categories)
output_bias = np.zeros(n_categories) + 0.01

In [84]:
print(hidden_weights.shape)
print(hidden_bias.shape)
print(output_weights.shape)
print(output_bias.shape)
print(X.shape)

(91, 50)
(50,)
(50, 2)
(2,)
(30000, 91)


In [82]:
z_h = np.matmul(X, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_o = np.matmul(a_h, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

In [88]:
print(X.shape)
print(z_h.shape)
print(a_h.shape)
print(z_o.shape)
print(probabilities.shape)

(30000, 91)
(30000, 50)
(30000, 50)
(30000, 2)
(30000, 2)


In [100]:
import numpy as np
fourbyfour = np.array([
                       [1,2,3,4],
                       [3,2,1,4],
                       [5,4,6,7],
                      ])


twobyfourbythree = np.array([
                             [[2,3],[11,9],[32,21],[28,17]],
                             [[2,3],[1,9],[3,21],[28,7]],
                             [[2,3],[1,9],[3,21],[28,7]],
                            ])

print(np.dot(fourbyfour,fourbyfour.T).shape)
print(np.matmul(fourbyfour,twobyfourbythree).shape)

(3, 3)
(3, 3, 2)
